<a href="https://colab.research.google.com/github/gmauricio-toledo/matematicas-aprendizaje-automatico/blob/main/Notebooks/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/notebooks/01-MLP-MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



✅ Cuando se entrenan redes neuronales es recomendable conectar la notebook en modo GPU

Entorno de ejecución → Cambiar tipo de entorno de ejecución

Algunas consideraciones:

* No dejar la notebook conectada sin actividad ya que Colab penaliza esto al asignar un entorno con GPU.
* No pedir el entorno con GPU si no se va a usar.

Recuerda la simbología de las secciones:

* 🔽 Esta sección no forma parte del proceso usual de Machine Learning. Es una exploración didáctica de algún aspecto del funcionamiento del algoritmo.
* ⚡ Esta sección incluye técnicas más avanzadas destinadas a optimizar o profundizar en el uso de los algoritmos.
* ⭕ Esta sección contiene un ejercicio o práctica a realizar. Aún si no se establece una fecha de entrega, es muy recomendable realizarla para practicar conceptos clave de cada tema.

# Ejemplo ilustrativo 1: Clasificación en $\mathbb{R}^2$

En este ejemplo veremos una tarea de clasificación y, por medio de las capas ocultas, veremos el proceso de extración de features de una red neuronal

Función para generar puntos en brazos de espirales en $\mathbb{R}^2$

In [ ]:
import numpy as np

def make_spiral_dataset(n_classes=2, points_per_arm=100,
                        noise=0.1, screw=0.01,
                        gap=0.1,
                        radius=1, num_arms=6):
    X = np.zeros((num_arms*points_per_arm,2),dtype=np.float32)
    y = np.zeros((num_arms*points_per_arm,),dtype=np.int32)

    for k in range(num_arms):
        rs = np.sort(np.random.uniform(0,radius,size=points_per_arm)) + gap
        theta = 0.5 + 2*k*np.pi/num_arms
        increments = np.array([screw*k for k in range(points_per_arm)])
        thetas = theta + increments + np.random.uniform(0,noise,size=points_per_arm)
        X[k*points_per_arm:(k+1)*points_per_arm,:] = np.array([rs*np.cos(thetas),rs*np.sin(thetas)]).T
        y[k*points_per_arm:(k+1)*points_per_arm] = np.array([k%n_classes]*points_per_arm)

    return X, y

## Ejemplo 1

In [ ]:
X, y = make_spiral_dataset(points_per_arm=400,
                           noise=0.3,
                           screw=0.01,
                           radius=2,
                           gap=0.2,
                           num_arms=6,
                           n_classes=6
                           )

plt.figure()
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.5)
plt.axis('equal')
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

y_classes = to_categorical(y,num_classes=6)

# # 1. PREPROCESAMIENTO
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# 2. Split manual con stratify (más control)
X_train, X_val, y_train, y_val = train_test_split(
    X, y_classes, test_size=0.2, random_state=42, stratify=y
)

# 3. Red balanceada: capacidad suficiente + regularización moderada
model = Sequential([
    Input(shape=(2,)),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(32, activation='relu'),
    Dense(8, activation='relu'),
    Dense(2, activation='linear', name='bottleneck_2d'),  # Cuello de botella
    Dense(6, activation='softmax')
])

# 4. Optimizer con learning rate controlado
optimizer = Adam(learning_rate=0.0005)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

# 5. Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    )
]

# 6. Entrenamiento
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

loss_train = history.history['loss']
loss_val = history.history['val_loss']
acc_train = history.history['accuracy']
acc_val = history.history['val_accuracy']
epochs = range(1,len(loss_train)+1)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Loss')
plt.xlabel("Época")
plt.ylabel("Loss")
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='validation accuracy')
plt.title('Accuracy')
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_val, y_val)

In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt

# Modelo para extraer las features 2d de la penúltima capa
feature_model = Model(inputs=model.layers[0].input,
                      outputs=model.get_layer('bottleneck_2d').output)
features_2d = feature_model.predict(X_train)

plt.figure(figsize=(15, 8))
plt.subplot(1,2,1)
for i in range(6):
    plt.scatter(X_train[y_train[:,i]==1, 0],
                X_train[y_train[:,i]==1, 1],
                label=f"Clase {i}")
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.legend()
plt.subplot(1,2,2)
for i in range(6):
    plt.scatter(features_2d[y_train[:,i]==1, 0],
                features_2d[y_train[:,i]==1, 1],
                label=f"Clase {i}")
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.legend()
plt.show()


## Ejemplo 2

In [ ]:
import matplotlib.pyplot as plt

X, y = make_spiral_dataset(points_per_arm=400,
                           noise=0.12,
                           screw=0.01,
                           radius=2,
                           num_arms=8,
                           n_classes=2
                           )
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

Definimos una red neuronal

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# # 1. PREPROCESAMIENTO
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# 2. Split con stratify
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Modelo MLP
model = Sequential([
    Input(shape=(2,)),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(16, activation='relu',name='features_16d'),
    Dense(1, activation='sigmoid')
])

# 4. Optimizer con learning rate controlado
optimizer = Adam(learning_rate=0.0005)

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

# 5. Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    )
]

# 6. Entrenamiento
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

loss_train = history.history['loss']
loss_val = history.history['val_loss']
acc_train = history.history['accuracy']
acc_val = history.history['val_accuracy']
epochs = range(1,len(loss_train)+1)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Loss')
plt.xlabel("Época")
plt.ylabel("Loss")
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='validation accuracy')
plt.title('Accuracy')
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_val, y_val)

Definimos el modelo que extrae las features de la última capa oculta

In [ ]:
from keras.models import Model

# Extrae features de 16D
feature_model = Model(inputs=model.layers[0].input,
                      outputs=model.get_layer('features_16d').output)
features_16d = feature_model.predict(X_train)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from umap import UMAP
import matplotlib.pyplot as plt

# Reduce a 2D para visualizar
pca = PCA(n_components=2)
features_2d_pca = pca.fit_transform(features_16d)

umap = UMAP(n_components=2)
features_2d_umap = umap.fit_transform(features_16d)

# Graficamos

plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.scatter(features_2d_pca[y_train==0, 0], features_2d_pca[y_train==0, 1],
            c='blue', alpha=0.6, label='Clase 0')
plt.scatter(features_2d_pca[y_train==1, 0], features_2d_pca[y_train==1, 1],
            c='red', alpha=0.6, label='Clase 1')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Features 16D proyectadas a 2D con PCA')
plt.legend()
plt.subplot(1, 2, 2)
plt.scatter(features_2d_umap[y_train==0, 0], features_2d_umap[y_train==0, 1],
            c='blue', alpha=0.6, label='Clase 0')
plt.scatter(features_2d_umap[y_train==1, 0], features_2d_umap[y_train==1, 1],
            c='red', alpha=0.6, label='Clase 1')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.title('Features 16D proyectadas a 2D con UMAP')
plt.legend()
plt.show()

# Ejemplo Ilustrativo 2: Aproximando funciones $f:\mathbb{R}^n\rightarrow\mathbb{R}^m$

Bajamos una función para graficar con plotly en 3D en HTML

In [ ]:
!wget -q https://raw.githubusercontent.com/gmauricio-toledo/tda-gdl/main/scatterplot3dHTML.py

from scatterplot3dHTML import scatter_plot_3d_plotly

Definamos una función $f:\mathbb{R}^2\rightarrow\mathbb{R}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import sin, cos

# EJEMPLO 1: Gaussiana modulada con oscilación radial
# def f(x, y):
#     r = np.sqrt(x**2 + y**2)
#     return np.exp(-r**2 / 4) * np.cos(2*r)

# EJEMPLO 2: Egg Crate
def f(x, y):
    return sin(x) * sin(y) + 0.1*(x**2 + y**2)

Generamos el conjunto de datos sampleando puntos sobre la superficie $z=f(x,y)$

In [ ]:
# Generar dataset
N = 5000
X = np.zeros((N, 2))
Y = np.zeros(N)

for i in range(N):
    x = np.random.uniform(-4, 4)
    y = np.random.uniform(-4, 4)
    X[i, :] = [x, y]
    Y[i] = f(x, y)

# Graficar el dataset
puntos3d = np.array([X[:, 0], X[:, 1], Y]).T
print(puntos3d.shape)

scatter_plot_3d_plotly(X=puntos3d,
                       filename='MLP-aproximation-dataset (Egg Crate).html',
                       fig_title='MLP aproximation of a function\nEgg Crate')

Dividimos en entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

def crear_mlp(input_dim=2, hidden_units=64, n_hidden_layers=2, output_dim=1,
              activation='tanh'):
    """
    Crea un MLP para aproximación de funciones

    Args:
        input_dim: dimensión de entrada (2 para f:R^2->R)
        hidden_units: número de neuronas por capa oculta
        n_hidden_layers: número de capas ocultas
        output_dim: dimensión de salida (1)
        activation: función de activación σ (debe ser no-polinomial)
    """
    model = Sequential()

    model.add(Input(shape=(input_dim,)))
    # Primera capa oculta
    model.add(Dense(hidden_units, activation=activation))

    # Capas ocultas adicionales
    for _ in range(n_hidden_layers - 1):
        model.add(Dense(hidden_units, activation=activation))

    # Capa de salida (sin activación, según la teoría del MLP)
    model.add(Dense(output_dim, activation='linear'))

    return model

# Crear el modelo
model = crear_mlp(input_dim=2, hidden_units=64, n_hidden_layers=2)

# Compilar
model.compile(
    optimizer='adam',
    loss='mse',  # Mean Squared Error para regresión
    metrics=['mae']  # Mean Absolute Error
)

# Ver arquitectura
model.summary()

Compilamos y entrenamos

In [ ]:
model.compile(
    loss='mse',
    optimizer='adam',
    metrics=['mae']
)

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=64,
    verbose=1,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt

loss_train = history.history['loss']
loss_val = history.history['val_loss']
acc_train = history.history['mae']
acc_val = history.history['val_mae']
epochs = range(1,len(loss_train)+1)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Loss')
plt.xlabel("Época")
plt.ylabel("Loss")
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, acc_train, 'g', label='Training MAE')
plt.plot(epochs, acc_val, 'b', label='validation MAE')
plt.title('Accuracy')
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

Hacemos un grid para graficar la superficie encontrada

In [ ]:
import numpy as np

# Hacemos el conjunto de datos de entrenamiento {(x,y,f(x,y))}
train_dataset = np.concatenate((X_train,y_train.reshape(-1,1)),axis=1)

# Hacemos un grid de [-4,4]x[-4,4] y evaluamos la función obtenida en cada punto
x1, x2 = np.meshgrid(np.linspace(-4,4,100),np.linspace(-4,4,100))
y_pred = model.predict(np.array([x1.flatten(),x2.flatten()]).T)
grid_dataset = np.concatenate((np.array([x1.flatten(),x2.flatten()]).T,y_pred),axis=1)

# Juntamos el dataset de entrenamiento y la superficie aproximada
total_dataset = np.concatenate((train_dataset,grid_dataset))
print(total_dataset.shape)
# Añadimos etiquetas para identificar los puntos de entrenamiento y la superficie obtenida
labels = np.zeros(shape=(total_dataset.shape[0],))
labels[:train_dataset.shape[0]] = 1
print(labels.shape)

Graficamos y guardamos en HTML

In [ ]:
hover_info = ['Train dataset' if k==1 else 'Aproximation' for k in labels]

scatter_plot_3d_plotly(X=total_dataset,
                       y=labels,
                       hover_info=hover_info,
                       filename='MLP-aproximation (Egg Crate).html',
                       fig_title='MLP aproximation of a function\nEgg Crate')

# Redes Neuronales MLP para clasificación

<img align="center" width="50%" src="https://github.com/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/img/mlp.png?raw=1"/>

Ahora usaremos una red neuronal de tipo **MultiLayer Perceptron (MLP)** para el problema de clasificación en el dataset MNIST.

Veremos que con una red neuronal podremos entrenar un modelo para la clasificación de este dataset usando todas las features sin mucho problema.

---

Recordemos el mejor modelo de Machine Learning clásico que hemos obtenido:

<center>
<p><img src="https://drive.google.com/uc?id=1XY3iypQcSR868H7xFBilyxuzsJTzed4g" width="500" /></p>
</center>

Este modelo era difícil de entrenar en cuestión de la duración del entrenamiento y requirió una busqueda exhaustiva de parámetros.

Con PCA (50-60 componentes principales) y SVM estabamos alrededor de 97% en las métricas.

Benchmarks para el dataset MNIST

1. **No Routing Needed Between Capsules**, 2020. *Accuracy: 99.87%*

    Modelo de redes CNN con Homogeneous Vector Capsules (HVCs) que modifican el flujo de datos entre capas. [Artículo](https://arxiv.org/abs/2001.09136), [código](https://github.com/AdamByerly/BMCNNwHFCs).

2. **An Ensemble of Simple Convolutional Neural Network Models for MNIST Digit Recognition**, 2020. *Accuracy: 99.87%*

    Modelo de ensamble de redes CNN [Artículo](https://arxiv.org/abs/2008.10400), [código](https://github.com/ansh941/MnistSimpleCNN).

## 1. El conjunto de datos

Observar que, ahora sí, usamos todo el conjunto de datos completo.

In [ ]:
import numpy as np
from keras.datasets import mnist

# Load MNIST handwritten digit data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

y_test_original = y_test.copy()  # Hacemos una copia del 'y_test', la usaremos al final

Recordar que tenemos que dividir en tres conjuntos de entrenamiento: train, validation y test.

<center>
<img src="https://drive.google.com/uc?export=view&id=1Ts1ZZlHl1qTdkD_FZ2uvi9wkJtuTNH8M" style="width: 60%">
</center>

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.15,
                                                  stratify=y_train,  # RECORDAR LA ESTRATIFIACIÓN
                                                  random_state=894)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

Veamos que esto no afecta a las imagenes:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Escogemos algunos índices:
idxs = np.random.choice(range(X_train.shape[0]),size=3,replace=False)

# Hacemos 3 tipos de reescalamiento:
X_minmax = MinMaxScaler().fit_transform(X_train[idxs,:].reshape(-1,28*28))
X_std = StandardScaler().fit_transform(X_train[idxs,:].reshape(-1,28*28))
X_scl = X_train[idxs,:].reshape(-1,28*28)/255.

# Títulos para cada fila
row_titles = [
    "Imágenes originales",
    "MinMaxScaler (por feature)",
    "StandardScaler (por feature)",
    "Reescalado [0, 1] (x/255)"
]

# Graficamos:
fig, axs = plt.subplots(ncols=3, nrows=4, sharex=False,
                       sharey=True, figsize=(6, 9))
for i, idx in enumerate(idxs):
	# Fila 0: Originales
	axs[0,i].imshow(X_train[idx], cmap='plasma')
	axs[0,i].axis('off')

	# Fila 1: MinMax
	axs[1,i].imshow(X_minmax[i].reshape(28,28), cmap='plasma')
	axs[1,i].axis('off')

	# Fila 2: StandardScaler
	axs[2,i].imshow(X_std[i].reshape(28,28), cmap='plasma')
	axs[2,i].axis('off')

	# Fila 3: Reescalado manual
	axs[3,i].imshow(X_scl[i].reshape(28,28), cmap='plasma')
	axs[3,i].axis('off')
	if i == 1:
		for j in range(4):
			axs[j,i].set_title(row_titles[j],fontsize=10)

fig.show()

🔵 Esto nos da una idea del data leakage

Apliquemos el preprocesamiento para este dataset:

In [ ]:
print(f"Máximo valor de X_train: {np.max(X_train)}")
print(f"Mínimo valor de X_train: {np.min(X_train)}")

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_val /= 255
X_test /= 255

print(f"Máximo valor de X_train: {np.max(X_train)}")
print(f"Mínimo valor de X_train: {np.min(X_train)}")

Visualizamos 6 ejemplos aleatorios, junto con sus etiquetas

In [ ]:
import matplotlib.pyplot as plt

# ------ Obtenemos algunos índices aleatorios:
some_idxs = np.random.choice(list(range(y_train.shape[0])),size=6,replace=False)

fig, axes = plt.subplots(ncols=6, sharex=False,
			 sharey=True, figsize=(10, 4))
for i,idx in enumerate(some_idxs):
	axes[i].set_title(y_train[idx],fontsize=15)
	axes[i].imshow(X_train[idx], cmap='gray')
	axes[i].get_xaxis().set_visible(False)
	axes[i].get_yaxis().set_visible(False)
plt.show()

## Definiendo la red

### Etiqueta de clase vs Vector de clase

**IMPORTANTE❗**

Al usar redes neuronales, usalmente el vector de etiquetas debe estar codificado como vectores **one-hot**. Es decir:

$$1 → (1,0,...,0) $$
$$2 → (0,1,...,0) $$
$$ ... $$

Entonces, las etiquetas $y$ son matrices de tamaño $N\times m$ donde

* $N$: número de instancias
* $m$: número de clases



Hacemos la codificación usando la función [`to_categorical`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) de [keras](https://www.tensorflow.org/guide/keras).

$$y_j \overset{\text{to_categorical}}{\rightarrow} (0,...,0,\overset{j}{1},0,...,0)$$

$$y_j \overset{\text{numpy.argmax}}{\leftarrow} (0,...,0,\overset{j}{1},0,...,0)$$

⚠ A lo largo de las versiones, a veces cambia de ubicación este tipo de funciones.





In [ ]:
from tensorflow.keras.utils import to_categorical

print("---------- Antes de la codificación ----------")
print(f"Primeras 5 etiquetas: {y_train[:5]}")
print(f"Shape: {y_train.shape}")

y_train = to_categorical(y_train,num_classes=10)

print("---------- Después de la codificación ----------")
print(f"Primeras 5 etiquetas:\n{y_train[:5]}\n")
print(f"Shape: {y_train.shape}")

y_val = to_categorical(y_val,num_classes=10)
y_test = to_categorical(y_test,num_classes=10)

### Ingredientes de una arquitectura

Hay dos principales maneras de definir modelos en Keras:

<center>
<img src="https://drive.google.com/uc?export=view&id=1lbLQ7D1_QElq_iTscpQ_rjXPWJ2uPke3" width=500>
</center>


* [**Sequential**](https://keras.io/api/models/sequential/): Un modelo es una secuencia lineal de *layers*. Es sencilla de implementar pero no muy flexible.


* [**Model**](https://www.tensorflow.org/api_docs/python/tf/keras/Model): Un modelo se especifica mediante una estructura similar a un *grafo*, indicando conexiones entre *layers*. Es muy flexible.



In [ ]:
from keras.models import Sequential

Por ahora, usaremos los siguientes tipos básicos de capas:

* [**Dense**](https://keras.io/api/layers/core_layers/dense/): implementa la operación:
$$\text{output} = \text{activation}(\text{input}\cdot\text{weights} + \text{bias})$$
donde `activation` es la función de activación y bias es un vector de sesgo creado por la capa (sólo aplicable si `use_bias` es True). Es una capa **densa** por lo que cada neurona de esta capa se conecta con cada una de las neuronas de la capa anterior.
* [**Flatten**](https://keras.io/api/layers/reshaping_layers/flatten/): Aplana los datos para tener un arreglo unidimensional.
* [**Input**](https://keras.io/api/layers/core_layers/input/): Define el tensor de entrada de la red con su forma. Es el punto de entrada para los datos en el modelo.

In [ ]:
from keras.layers import Dense, Flatten, Input

Definimos ahora la arquitectura de la red neuronal MLP. Observa los siguientes elementos:

* La **función de activación** de cada capa, [documentación](https://keras.io/api/layers/activations/).
* La **función de perdida** de la red, es la función de costo que mide que tanto error hay en las predicciones. El optimizador minimizará está función, [documentación](https://keras.io/api/losses/).
* El **optimizador** es la clase que minimizará la función de perdida. De su elección depende qué tan rápido converjamos a una solución, [documentación](https://keras.io/api/optimizers/)
* La(s) **métrica(s) de desempeño** a monitorear durante el entrenamiento, tanto en el conjunto de entrenamiento como en el de validación. Además, podemos evaluar las métricas usuales al generar las predicciones. [Documentación](https://keras.io/api/metrics/)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input

model = Sequential()
model.add(Input(shape=(28,28)))
model.add(Flatten()) # Tenemos que aplanar las matrices representando a cada imagen, las capas densas sólo funcionan con vectores de entrada
model.add(Dense(8, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Cuando se trata de tareas de clasificación multiclase, ponemos una activación softmax en la capa de salida

In [ ]:
model.summary()

El método `summary()` de un modelo de Keras (ya sea `Sequential` o `Model`) imprime información importante del modelo. [Documentación](https://keras.io/api/models/model/).

El siguiente paso es compilar el modelo. Esto lo hacemos con el método [`compile`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile).

---

Una guía general sobre funciones de perdida y métricas

| Aplicación                     | Función de Pérdida (Loss)          | Métrica usual           | Última capa (output layer)          |
|---------------------------------|------------------------------------|-------------------------|-------------------------------------|
| Clasificación binaria          | `binary_crossentropy`              | `accuracy`              | `Dense(1, activation='sigmoid')`    |
| Clasificación multiclase       | `categorical_crossentropy`         | `accuracy`              | `Dense(num_clases, activation='softmax')` |
| Regresión (un valor)           | `mean_squared_error` (MSE)         | `mse` o `mae`           | `Dense(1, activation='linear')`     |
| Regresión (múltiples valores)  | `mean_squared_error` (MSE)         | `mse` o `mae`           | `Dense(num_valores, activation='linear')` |

Una guía general sobre optimizadores:

| Optimizador  | Ventajas                             | Casos de Uso Típicos         | Parámetros Clave               |
|--------------|--------------------------------------|------------------------------|--------------------------------|
| **Adam**     | Convergencia rápida, adaptable      | Default para MLPs, CNN, RNN  | `lr`  |
| **SGD**      | Mayor control, estable con momentum | Problemas convexos, fine-tuning | `lr`, `momentum`      |
| **RMSprop**  | Bueno para datos ruidosos           | RNNs, problemas inestables    | `lr`, `rho`          |

---

⚠ ❗ **IMPORTANTE** Antes de volver a entrenar un modelo, **debes recompilarlo** con el método `compile()`. Esto reinicializa los pesos aleatoriamente y evita que el entrenamiento continúe desde los pesos previamente aprendidos.  

In [ ]:
model.compile(loss='categorical_crossentropy',
	      optimizer='adam',
	      metrics=['acc']
		  )

## Entrenando la red

Entrenamos la red con el método [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), la mécanica es la misma que en los métodos de ML clásico.

Algunas diferencias son:

* Especificar el número de épocas. Entre más épocas, más puede aprender el módelo, aunque hay más riesgo de overfitting.

* Especificar el conjunto de validación, además del conjunto de entrenamiento. Este sirve para proporcionar un indicador no sesgado del desempeño del modelo. Se puede hacer especificamente, o como una fracción del conjunto de entrenamiento.




Observa las métricas y pérdida en el conjunto de entrenamiento y validación.

El entrenamiento regresa un objeto de tipo `History`. Su atributo History.history es un registro de valores de pérdidas de entrenamiento y valores de métricas en épocas sucesivas, así como valores de pérdidas de validación y valores de métricas de validación (si procede).

In [ ]:
n_epocas = 8

history = model.fit(X_train, y_train, epochs=n_epocas, validation_data=(X_val,y_val))

Graficamos la función de perdida en cada época, tanto en el conjunto de entrenamiento, como en el de validación.

Estas se llaman **gráficas de entrenamiento**. Son muy importantes para evaluar si hay overfitting, entre otras cosas.

Observa que los registros *históricos* del entrenamiento (perdidas y métricas) se encuentran en el diccionario `history.history` especificado anteriormente.

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
acc_train = history.history['acc']
acc_val = history.history['val_acc']

epochs = range(1,n_epocas+1)

fig, axs = plt.subplots(1,2,figsize=(12,5))
axs[0].plot(epochs, loss_train, 'g', label='Training loss')
axs[0].plot(epochs, loss_val, 'b', label='validation loss')
axs[0].title.set_text('Loss')
axs[0].set(xlabel="Época", ylabel="Loss")
axs[0].legend()
axs[1].plot(epochs, acc_train, 'g', label='Training accuracy')
axs[1].plot(epochs, acc_val, 'b', label='validation accuracy')
axs[1].title.set_text('Accuracy')
axs[1].set(xlabel="Época", ylabel="Accuracy")
axs[1].legend()

fig.show()

🔵 ¿Hay señales de overfitting? ¿underfitting?

⚡ De la siguiente forma podemos acceder a la matriz de pesos y sesgos en cada capa. Las guardamos como arreglos de numpy.

In [ ]:
first_layer_weights = model.layers[1].get_weights()[0]
first_layer_biases  = model.layers[1].get_weights()[1]

np.save("mnist_weights1.npy",first_layer_weights)
np.save("mnist_biases1.npy",first_layer_biases)

In [ ]:
second_layer_weights = model.layers[2].get_weights()[0]
second_layer_biases  = model.layers[2].get_weights()[1]

np.save("mnist_weights2.npy",second_layer_weights)
np.save("mnist_biases2.npy",second_layer_biases)

## Predicciones y rendimiento

🔽 ¿Cómo se ven las predicciones?

In [ ]:
from seaborn import heatmap
import numpy as np
import matplotlib.pyplot as plt

# ----- Escogemos un elemento del conjunto test:
idx = np.random.choice(range(X_test.shape[0]),size=1)[0]
print(f"índice test: {idx}")
x = X_test[idx].copy()

# ----- Graficamos este ejemplo de prueba:
plt.figure()
plt.suptitle(y_test_original[idx],fontsize=15)
plt.imshow(x, cmap='gray')
plt.xticks([])
plt.yticks([])
plt.show()

# ----- Cambiamos a la forma adecuada para entrar a la red neuronal:
x_input = x.reshape(-1,x.shape[0],x.shape[1])

# ----- Lo pasamos por la red neuronal ya entrenada:
prediction = model.predict(x_input)
print(f"\nSalida de la red neuronal para este elemento:")

plt.figure(figsize=(5,1))
heatmap(prediction, cmap='plasma',annot=np.round(prediction,2),cbar=False)
plt.xticks([])
plt.yticks([])
plt.show()

print(f"Son probabilidades, la suma de las entradas es {np.sum(prediction)}")

# ----- Tomamos el argmax:
prediction = np.argmax(prediction, axis=1)
print(f"\nTomamos el índice de la entrada con mayor probabilidad: {prediction}")

Obtenemos todas las predicciones sobre el conjunto de prueba.

In [ ]:
predictions_matrix = model.predict(X_test,batch_size=1)  # Observa el batch_size
predictions = np.argmax(predictions_matrix, axis=1)  # Prueba a comentar esta línea y discutamos qué pasa

Visualizamos algunas predicciones

In [ ]:
idxs = np.random.choice(range(X_test.shape[0]),size=10,replace=False)

fig, axes = plt.subplots(ncols=10, sharex=False,
			 sharey=True, figsize=(18, 4))
for i,idx in enumerate(idxs):
	axes[i].set_title(predictions[idx])
	axes[i].imshow(X_test[idx], cmap='gray')
	axes[i].get_xaxis().set_visible(False)
	axes[i].get_yaxis().set_visible(False)
plt.show()

Obtenemos las métricas de desempeño de la tarea de clasificación. Observar que ambas son **vectores** de etiquetas

In [ ]:
print(predictions.shape)
print(y_test_original.shape)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

print(f"Test Accuracy: {accuracy_score(y_pred=predictions,y_true=y_test_original)}")
print(f"Test Recall: {recall_score(y_pred=predictions,y_true=y_test_original,average='macro')}")
print(f"Test Precision: {precision_score(y_pred=predictions,y_true=y_test_original,average='macro')}")

Calculamos el roc-auc score

In [ ]:
from sklearn.metrics import roc_auc_score

print(f"Shape de y_test: {y_test.shape}")
print(f"Shape de las predicciones para el conjunto de prueba: {predictions_matrix.shape}")

ra_score = roc_auc_score(y_test,predictions_matrix)
print(f"ROC-AUC score {ra_score}")

Mostramos la matriz de confusión

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

plt.figure()
cm = confusion_matrix(y_test_original,predictions)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

⭕ ¿Qué dígitos son los que más confunde la red?

La diferencia entre las métricas de clasificación (accuracy, precision, recall) y el ROC-AUC radica en lo que cada una evalúa. Las métricas de clasificación miden el rendimiento del modelo con un umbral de decisión fijo, típicamente 0.5 para clasificación binaria o el valor máximo de probabilidad para multiclase. En contraste, el ROC-AUC evalúa la capacidad del modelo para distinguir entre clases a través de todos los posibles umbrales de decisión. Un modelo puede tener un ROC-AUC alto porque ordena correctamente las muestras según su probabilidad de pertenencia a cada clase, pero mostrar métricas de clasificación moderadas porque las probabilidades predichas no están calibradas para el umbral de decisión utilizado. Esta discrepancia indica que el modelo posee buena capacidad discriminativa pero requiere optimización del umbral de decisión o calibración de probabilidades para maximizar su rendimiento en la clasificación final.

### 🔽 Predicción en el mundo real

Observa que este modelo está listo para hacer predicciones en el *mundo real*. Carguemos algunas imagenes de dígitos hechos por tí:

* Escribe un dígito en una hoja
* Tómale foto
* Guardalo como imagen png
* Súbela a colab
* Ejecuta la celda siguiente con las modificaciones pertinentes

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def procesar_imagen(ruta_imagen):
    img = Image.open(ruta_imagen) # Leer la imagen
    img_gris = img.convert('L') # Convertir a escala de grises
    img_28x28 = img_gris.resize((28, 28),
                                # Image.Resampling.LANCZOS
                                ) # Redimensionar a 28x28 píxeles
    arreglo = np.array(img_28x28, dtype=int) # Convertir a arreglo NumPy
    arreglo = np.clip(arreglo, 0, 255) # Asegurar que los valores están en 0-255
    return arreglo


ruta = '/content/digito0.jpeg'  # Cambia por tu ruta

arreglo_28x28 = procesar_imagen(ruta)
arreglo_28x28 = 255 - arreglo_28x28  # Invierte colores
arreglo_28x28 = arreglo_28x28.astype('int')/255

plt.figure(figsize=(4,4))
plt.imshow(arreglo_28x28, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
print(arreglo_28x28)

In [ ]:
prediccion = model.predict(arreglo_28x28.reshape(-1,28,28))
np.argmax(prediccion)

---

## ⭕ Práctica y Ejercicios

En esta práctica vamos a definir, compilar y entrenar varias arquitecturas de redes neuronales

Implementa las siguientes redes neuronales de tipo MLP:

* 1 capa oculta de 200 neuronas sin activación. Entrena durante 30 épocas.
* 1 capa oculta de 200 neuronas con activación $tanh$. Entrena durante 30 épocas.
* 3 capas ocultas de 100, 200 y 100 neuronas respectivamente, todas con activación ReLU. Entrena durante 50 épocas.

En cada uno de los experimentos determina las especificaciones de las capas de entrada y salida. Además, en cada caso, reporta el accuracy y recall en el conjunto de prueba, así como las curvas de entrenamiento (perdida y accuracy).

* Con el objetivo de subir la métrica de accuracy en el conjunto de prueba, entrena un nuevo módelo de red neuronal MLP cambiando los siguientes hiperparámetros:

 * Número de capas ocultas.
 * Número de neuronas en cada capa oculta.
 * Función de activación de cada capa oculta.
 * Optimizador ([opciones](https://keras.io/api/optimizers/)).


---


Como referencia, el mejor resultado hasta ahora, sin usar redes convolucionales, es un accuracy de 99.65% (https://arxiv.org/abs/1003.0358)

Lista de resultados: http://yann.lecun.com/exdb/mnist/, https://paperswithcode.com/sota/image-classification-on-mnist

🟢 Descargamos el conjunto de datos y preprocesamiento

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.15,
                                                  stratify=y_train,
                                                  random_state=782)

X_train = X_train.astype('float32')/255
X_val = X_val.astype('float32')/255
X_test = X_test.astype('float32')/255

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


y_test_original = y_test.copy()
y_train = to_categorical(y_train,num_classes=10)
y_val = to_categorical(y_val,num_classes=10)
y_test = to_categorical(y_test,num_classes=10)

Define una arquitectura con una capa oculta de 200 neuronas, con activación `tanh`

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input

model = Sequential()
model.add(Input(shape=(28,28)))
model.add(Flatten())

# añade la capa oculta:


# ---------------

model.add(Dense(10, activation='softmax'))

# Imprimimos el summary:
model.summary()

🔴 Compila el modelo con la misma función de perdida, métrica y optimzador

🔴 Entrena el modelo en el conjunto de entrenamiento, usa el conjunto `X_val, y_val` como validación. Entrena durante 15 épocas.

🟢 Graficamos la pérdida y métrica durante el entrenamiento, ¿observas overfitting o underfitting?

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
acc_train = history.history['acc']
acc_val = history.history['val_acc']

epochs = range(1,n_epocas+1)

fig, axs = plt.subplots(1,2,figsize=(12,5))
axs[0].plot(epochs, loss_train, 'g', label='Training loss')
axs[0].plot(epochs, loss_val, 'b', label='validation loss')
axs[0].title.set_text('Loss')
axs[0].set(xlabel="Época", ylabel="Loss")
axs[0].legend()
axs[1].plot(epochs, acc_train, 'g', label='Training accuracy')
axs[1].plot(epochs, acc_val, 'b', label='validation accuracy')
axs[1].title.set_text('Accuracy')
axs[1].set(xlabel="Época", ylabel="Accuracy")
axs[1].legend()

fig.show()

🔴 Obten las predicciones y evalua el desempeño del modelo usando F1-score y Accuracy. Imprime ambas métricas

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

predictions_matrix = ...
y_pred = np.argmax(predictions_matrix, axis=1)

accuracy = ...
f1score = ...

🔴 En una sola celda, define una red MLP con las siguientes especificaciones:

* Capa de entrada (indica el `shape` adecuado)
* Capa Flatten
* Capa Oculta Densa de 200 neuronas, activación `relu`
* Capa Oculta Densa de 500 neuronas, activación `tanh`
* Capa Oculta Densa de 200 neuronas, activación `relu`
* Capa de Salida adecuada con activación adecuada

El resto de pasos como el ejercicio anterior.


### Solución: Un mejor modelo



A continuación se muestra una solución con un modelo entrenado previamente:

In [ ]:
import os

# URL del archivo raw en GitHub
url = "https://github.com/DCDPUAEM/DCDP/raw/main/04%20Deep%20Learning/data/mejor_modelo_mlp.h5"

# Descargar el archivo usando wget
!wget {url}

In [ ]:
from tensorflow import keras
import numpy as np

# Cargar el modelo
mejor_modelo = keras.models.load_model('mejor_modelo_mlp.h5')

# Ver arquitectura del modelo entrenado y cargado
print("=== Resumen del modelo ===")
mejor_modelo.summary()

In [ ]:
prediccion = mejor_modelo.predict(X_test.reshape(-1,28*28))
y_pred = np.argmax(prediccion, axis=1)
print(y_pred[:10])

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from seaborn import heatmap
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test_original,y_pred)
plt.figure()
heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.show()

print(f"Test Accuracy: {accuracy_score(y_pred=y_pred,y_true=y_test_original)}")
print(f"Test F1-score: {f1_score(y_pred=y_pred,y_true=y_test_original,average='macro')}")

In [ ]:
!wget https://github.com/gmauricio-toledo/matematicas-aprendizaje-automatico/raw/main/Galería/digito.jpeg

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def procesar_imagen(ruta_imagen):
    img = Image.open(ruta_imagen) # Leer la imagen
    img_gris = img.convert('L') # Convertir a escala de grises
    img_28x28 = img_gris.resize((28, 28),
                                # Image.Resampling.LANCZOS
                                ) # Redimensionar a 28x28 píxeles
    arreglo = np.array(img_28x28, dtype=int) # Convertir a arreglo NumPy
    arreglo = np.clip(arreglo, 0, 255) # Asegurar que los valores están en 0-255
    return arreglo


ruta = '/content/digito.jpeg'  # Cambia por tu ruta

arreglo_28x28 = procesar_imagen(ruta)
arreglo_28x28 = 255 - arreglo_28x28  # Invierte colores
arreglo_28x28 = arreglo_28x28.astype('int')/255

plt.figure(figsize=(4,4))
plt.imshow(arreglo_28x28, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
prediccion = mejor_modelo.predict(arreglo_28x28.reshape(-1,28*28))
print(f"Clase predicha: {np.argmax(prediccion)}")